In [1]:
!pip install -r requirements.txt
import pandas as pd
import numpy as np
import polars as pl
from datetime import datetime
import holidays
import time

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.metrics import mean_squared_error, f1_score, silhouette_score, calinski_harabasz_score, davies_bouldin_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.cluster import KMeans
import category_encoders as ce

import lightgbm as lgb
from tslearn.clustering import TimeSeriesKMeans, KShape
from tslearn.preprocessing import TimeSeriesScalerMeanVariance
from tslearn.metrics import cdist_dtw
from tslearn.barycenters import dtw_barycenter_averaging

import warnings
import requests
import os
import tqdm
from scipy.spatial import distance, KDTree

from google.cloud import bigquery
from google.oauth2 import service_account

warnings.simplefilter('ignore')

  Using cached lightgbm-3.3.2-py3-none-manylinux1_x86_64.whl (2.0 MB)
  Obtaining dependency information for holidays from https://files.pythonhosted.org/packages/5d/8c/bad7c11afc8969834728c0678bbf8f3ec5dba4c4ac7f5ad8cf91d63e865f/holidays-0.40-py3-none-any.whl.metadata
  Using cached holidays-0.40-py3-none-any.whl.metadata (21 kB)
  Using cached pandas_datareader-0.10.0-py3-none-any.whl (109 kB)
  Obtaining dependency information for tslearn from https://files.pythonhosted.org/packages/97/22/8dba9a7149d51fe0b6163a5a6b7efc315ab3c097cb6b0d1fc649a03f2722/tslearn-0.6.3-py3-none-any.whl.metadata
  Using cached tslearn-0.6.3-py3-none-any.whl.metadata (14 kB)
  Obtaining dependency information for category_encoders from https://files.pythonhosted.org/packages/7f/e5/79a62e5c9c9ddbfa9ff5222240d408c1eeea4e38741a0dc8343edc7ef1ec/category_encoders-2.6.3-py2.py3-none-any.whl.metadata
  Using cached category_encoders-2.6.3-py2.py3-none-any.whl.metadata (8.0 kB)
  Obtaining dependency information for

/opt/conda/lib/python3.10/site-packages/tslearn/bases/bases.py:15: UserWarning: h5py not installed, hdf5 features will not be supported.
Install h5py to use hdf5 features: http://docs.h5py.org/
  warn(h5py_msg)


In [2]:
df = pd.read_csv("df_includeID.csv")
df.drop('Unnamed: 0',axis=1,inplace=True)

In [3]:
df['date'] = pd.to_datetime(df['date'])
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['date'] = df['date'].dt.day

In [4]:
def createId(data):
    data['AccountID'] = data.groupby(['customer', 'lineItem_UsageAccountId', 'product_region', 'product_operatingSystem', 'product_instanceType']).ngroup()+1
    data['InstanceID'] = data.groupby(['customer','product_region', 'product_operatingSystem', 'product_instanceType']).ngroup()+1
    return data

In [5]:
df = createId(df)

In [6]:
grouped_df = df.groupby(['year', 'month', 'date', 'InstanceID'])['sum_num_machine'].sum().reset_index(name='total_sum_num_machine')

df = pd.merge(df, grouped_df, on=['year', 'month', 'date', 'InstanceID'], how='left')
df.drop('sum_num_machine',axis=1,inplace=True)
df.drop_duplicates(inplace=True)

In [7]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing

def byinstanceId(df,n):
    start_time = time.time()
    grouped_df = df.groupby(['year', 'month', 'date', 'InstanceID'])['total_sum_num_machine'].sum().reset_index()

    X = grouped_df[['total_sum_num_machine']]

    min_max_scaler = preprocessing.MinMaxScaler()
    X_scaled = min_max_scaler.fit_transform(X)

    kmeans = KMeans(n_clusters=n) 
    grouped_df['byinstanceId_cluster'] = kmeans.fit_predict(X_scaled)

    df = pd.merge(df, grouped_df[['year', 'month', 'date', 'InstanceID', 'byinstanceId_cluster']], 
                         on=['year', 'month', 'date', 'InstanceID'], 
                         how='left')
    print('クラスタリング完了しました')
    end_time = time.time()
    print(f'経過時間 : {end_time-start_time}')
    return df

In [8]:
def byAccountId(df,n):
    start_time = time.time()
    grouped_df = df.groupby(['year', 'month', 'date', 'AccountID'])['total_sum_num_machine'].sum().reset_index()

    X = grouped_df[['total_sum_num_machine']]

    min_max_scaler = preprocessing.MinMaxScaler()
    X_scaled = min_max_scaler.fit_transform(X)

    kmeans = KMeans(n_clusters=n) 
    grouped_df['byAccountID_cluster'] = kmeans.fit_predict(X_scaled)

    df = pd.merge(df, grouped_df[['year', 'month', 'date', 'AccountID', 'byAccountID_cluster']], 
                         on=['year', 'month', 'date', 'AccountID'], 
                         how='left')
    print('クラスタリング完了しました')
    end_time = time.time()
    print(f'経過時間 : {end_time-start_time}')
    return df

In [9]:
df = byAccountId(df,3)
df = byinstanceId(df,3)

クラスタリング完了しました
経過時間 : 5.850495100021362
クラスタリング完了しました
経過時間 : 1.7223103046417236


In [10]:
import time
import pandas as pd
from tslearn.clustering import TimeSeriesKMeans
from tslearn.utils import to_time_series_dataset

def byAccountId(df, n):
    start_time = time.time()

    time_series = []
    valid_account_ids = []
    for account_id, group in df.groupby('AccountID'):
        ts = group.sort_values(by=['year', 'month', 'date'])['total_sum_num_machine'].values
        if len(ts) > 1:
            time_series.append(ts)
            valid_account_ids.append(account_id)

    formatted_dataset = to_time_series_dataset(time_series)

    model = TimeSeriesKMeans(n_clusters=n, metric="dtw", verbose=True, max_iter_barycenter=5,n_jobs=-1)
    clusters = model.fit_predict(formatted_dataset)

    cluster_map = {account_id: cluster for account_id, cluster in zip(valid_account_ids, clusters)}

    df['byAccountID_cluster'] = df['AccountID'].map(cluster_map)

    print('クラスタリング完了しました')
    end_time = time.time()
    print(f'経過時間 : {end_time - start_time}')
    return df

In [11]:
df = byAccountId(df, 3)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    4.9s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:    5.7s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:    7.0s
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:    8.7s
[Parallel(n_jobs=-1)]: Done 1234 tasks      | elapsed:   10.9s
[Parallel(n_jobs=-1)]: Done 1784 tasks      | elapsed:   13.8s
[Parallel(n_jobs=-1)]: Done 2434 tasks      | elapsed:   17.2s
[Parallel(n_jobs=-1)]: Done 3184 tasks      | elapsed:   21.1s
[Parallel(n_jobs=-1)]: Done 4034 tasks      | elapsed:   25.3s
[Parallel(n_jobs=-1)]: Done 4379 out of 4379 | elapsed:   27.1s finished
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:    1.1s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:    2.4s
[Parallel(n_jobs

100405414.782 --> 

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:    2.4s
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:    4.1s
[Parallel(n_jobs=-1)]: Done 1234 tasks      | elapsed:    6.6s
[Parallel(n_jobs=-1)]: Done 1784 tasks      | elapsed:    9.2s
[Parallel(n_jobs=-1)]: Done 2434 tasks      | elapsed:   12.7s
[Parallel(n_jobs=-1)]: Done 3184 tasks      | elapsed:   16.7s
[Parallel(n_jobs=-1)]: Done 4034 tasks      | elapsed:   21.0s
[Parallel(n_jobs=-1)]: Done 4984 tasks      | elapsed:   26.7s
[Parallel(n_jobs=-1)]: Done 6034 tasks      | elapsed:   32.4s
[Parallel(n_jobs=-1)]: Done 7184 tasks      | elapsed:   38.9s
[Parallel(n_jobs=-1)]: Done 8434 tasks      | elapsed:   45.9s
[Parallel(n_jobs=-1)]: Done 9784 tasks      | elapsed:   53.4s
[Parallel(n_jobs=-1)]: Done 11234 tasks  

26589296.930 --> 

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:    2.4s
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:    4.0s
[Parallel(n_jobs=-1)]: Done 1234 tasks      | elapsed:    6.5s
[Parallel(n_jobs=-1)]: Done 1784 tasks      | elapsed:    9.2s
[Parallel(n_jobs=-1)]: Done 2434 tasks      | elapsed:   12.9s
[Parallel(n_jobs=-1)]: Done 3184 tasks      | elapsed:   17.0s
[Parallel(n_jobs=-1)]: Done 4034 tasks      | elapsed:   21.3s
[Parallel(n_jobs=-1)]: Done 4984 tasks      | elapsed:   26.8s
[Parallel(n_jobs=-1)]: Done 6034 tasks      | elapsed:   32.4s
[Parallel(n_jobs=-1)]: Done 7184 tasks      | elapsed:   38.6s
[Parallel(n_jobs=-1)]: Done 8434 tasks      | elapsed:   45.4s
[Parallel(n_jobs=-1)]: Done 9784 tasks      | elapsed:   52.9s
[Parallel(n_jobs=-1)]: Done 11234 tasks  

...

26171179.853 --> 

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:    2.4s
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:    4.1s
[Parallel(n_jobs=-1)]: Done 1234 tasks      | elapsed:    6.6s
[Parallel(n_jobs=-1)]: Done 1784 tasks      | elapsed:    9.4s
[Parallel(n_jobs=-1)]: Done 2434 tasks      | elapsed:   12.9s
[Parallel(n_jobs=-1)]: Done 3184 tasks      | elapsed:   16.9s
[Parallel(n_jobs=-1)]: Done 4034 tasks      | elapsed:   21.2s
[Parallel(n_jobs=-1)]: Done 4984 tasks      | elapsed:   26.6s
[Parallel(n_jobs=-1)]: Done 6034 tasks      | elapsed:   32.3s
[Parallel(n_jobs=-1)]: Done 7184 tasks      | elapsed:   38.6s
[Parallel(n_jobs=-1)]: Done 8434 tasks      | elapsed:   45.5s
[Parallel(n_jobs=-1)]: Done 9784 tasks      | elapsed:   53.1s
[Parallel(n_jobs=-1)]: Done 11234 tasks  

26171179.853 --> 


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:    2.3s
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:    4.0s
[Parallel(n_jobs=-1)]: Done 1234 tasks      | elapsed:    6.5s
[Parallel(n_jobs=-1)]: Done 1784 tasks      | elapsed:    9.3s
[Parallel(n_jobs=-1)]: Done 2434 tasks      | elapsed:   12.7s
[Parallel(n_jobs=-1)]: Done 3184 tasks      | elapsed:   16.7s
[Parallel(n_jobs=-1)]: Done 4034 tasks      | elapsed:   21.1s
[Parallel(n_jobs=-1)]: Done 4984 tasks      | elapsed:   26.6s
[Parallel(n_jobs=-1)]: Done 6034 tasks      | elapsed:   32.4s
[Parallel(n_jobs=-1)]: Done 7184 tasks      | elapsed:   38.7s
[Parallel(n_jobs=-1)]: Done 8434 tasks      | elapsed:   45.5s
[Parallel(n_jobs=-1)]: Done 9784 tasks      | elapsed:   53.1s
[Parallel(n_jobs=-1)]: Done 11234 tasks  

クラスタリング完了しました
経過時間 : 7280.313184499741
